In [1]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch import optim 
from torch import nn
from torch.utils.data import (
    DataLoader,
)
from tqdm import tqdm

In [2]:
class CNN(nn.Module):
    def __init__(self,in_channels = 1, num_classes = 10):
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(in_channels = in_channels, out_channels = 8, kernel_size = (3,3), stride = (1,1), padding = (1,1)) # Same Convolution
        # Formula => nout = (nin + 2*padding - kernel_size)/stride + 1
        # 28 = (28 + 2*1 - 3)/1 + 1
        self.pool = nn.MaxPool2d(kernel_size = (2,2), stride = (2,2))
        # 14 = (28 - 2)/2 + 1
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (3,3), stride = (1,1), padding = (1,1))
        self.fc1 = nn.Linear(16*7*7, num_classes)

    def forward(self,x):
        x = F.relu(self.conv1(x))
        print(x.shape)
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0],-1)
        x = self.fc1(x)
        
        return x

In [5]:
model = CNN()
x = torch.randn(64,1,28,28)
model(x)

torch.Size([64, 8, 28, 28])


tensor([[ 6.0755e-01,  2.8551e-01, -1.8305e-01, -4.1430e-02,  2.5681e-01,
          1.9053e-01,  2.9502e-01,  3.7516e-02, -2.9793e-01, -2.8718e-02],
        [ 4.2166e-01,  3.6591e-01, -2.2302e-01, -2.8643e-02,  1.7334e-01,
          1.0464e-01,  2.8438e-01,  1.3677e-01, -2.9036e-01,  1.5098e-01],
        [ 4.5310e-01,  3.4885e-01, -2.4003e-02, -9.7730e-02,  6.4442e-02,
          2.1208e-01,  3.3750e-01,  1.9098e-01, -3.3637e-02,  2.8714e-01],
        [ 5.7389e-01,  4.4351e-01, -1.3765e-01,  4.3192e-02,  2.2004e-01,
          1.8771e-01,  2.4651e-01,  7.1987e-02, -2.9906e-01,  1.4861e-01],
        [ 2.9999e-01,  3.6587e-01, -9.2368e-02, -5.1892e-03,  1.1081e-01,
          1.0634e-01,  2.5247e-01,  3.9271e-02, -1.0624e-01,  8.7364e-03],
        [ 4.1988e-01,  2.3914e-01, -1.6101e-01, -1.3824e-01,  1.8411e-01,
          1.3838e-01,  2.8210e-01,  6.2582e-02, -1.2425e-01,  5.2496e-02],
        [ 4.5116e-01,  5.1801e-01, -2.8300e-01, -1.0331e-01,  1.6716e-01,
          9.1583e-03,  1.7806e-0

In [4]:
# Code for Training and Testing the model

In [23]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
in_channels = 1
num_classes = 10
learning_rate = 3e-4
batch_size = 64
num_epochs = 3

# Load Data
train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)
test_dataset = datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True
)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# Initialize network
model = CNN(in_channels=in_channels, num_classes=num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:13<00:00, 731kB/s] 


Extracting dataset/MNIST\raw\train-images-idx3-ubyte.gz to dataset/MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 48.0kB/s]


Extracting dataset/MNIST\raw\train-labels-idx1-ubyte.gz to dataset/MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



...

Extracting dataset/MNIST\raw\t10k-images-idx3-ubyte.gz to dataset/MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 909kB/s]


Extracting dataset/MNIST\raw\t10k-labels-idx1-ubyte.gz to dataset/MNIST\raw



100%|██████████| 938/938 [00:06<00:00, 135.06it/s]


Accuracy on training set: 96.82
Accuracy on test set: 97.00
